In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import time
import sys
import datetime
import gc
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss
import lightgbm as lgb
import catboost as ctb
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2, SelectPercentile
import math
from sklearn.metrics import f1_score
import jieba
import jieba.posseg as psg
from collections import Counter
import functools
from time import time
train_data=pd.read_pickle("/home/kesci/work/first_zzp/train_feature_first_no_query_and_title_data.pickle")

In [2]:
test_data=pd.read_pickle("/home/kesci/work/first_zzp/test_feature_first_no_query_and_title_data.pickle")

FileNotFoundError: [Errno 2] No such file or directory: '/home/kesci/work/first_zzp/test_feature_first_no_query_and_title_data.pickle'

In [ ]:
train_data.shape,test_data.shape

In [ ]:
test_features = test_data[train_data.drop(['label','query_title_types'],axis = 1).columns]

In [ ]:
train_length = train_data.shape[0]
dataset = pd.concat([train_data.drop(['label','query_title_types'],axis = 1),test_features],axis = 0)
label = train_data['label']
del train_data,test_features
gc.collect()

In [ ]:
for column in ['title_ctr','title_query_cosine_similarity','title_sum','q_t_common_words',
                'title_query_dot_similarity','longistStr_len','0_lsi']:
    dataset[column+"_count"] = dataset.groupby(column)[column].transform("count")
    print(column+" has finished!")

In [ ]:
train_features = dataset[:train_length]
test_features = dataset[train_length:]

In [ ]:
train_features.shape,test_features.shape

In [ ]:
ctb_model = ctb.CatBoostClassifier(iterations=500,learning_rate=0.05,max_depth=11,l2_leaf_reg=1,verbose=10)

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
ss=StratifiedShuffleSplit(n_splits=1,test_size=0.2,train_size=0.8,random_state=0)
for train_index, test_index in ss.split(train_features, label):
    ctb_model.fit(train_features.iloc[train_index],label[train_index],eval_set=(train_features.iloc[test_index],label[test_index]))

In [18]:
import seaborn as sns
import matplotlib.pyplot as plt
importance = ctb_model.feature_importances_
columns = train_data.drop(['label','query_title_types'],axis = 1).columns
feature_importance_df = pd.concat([pd.DataFrame(importance),pd.DataFrame(columns)],axis = 1)
feature_importance_df.columns = ['importance','feature']
cols = (feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:1000].index)
best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]
plt.figure(figsize=(15, 0.2*test_features.shape[1]))
sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
plt.title('Catboost Features (One Folds)')
plt.show()   

<Figure size 1080x648 with 1 Axes>

In [26]:
best_features.sort_values(by="importance" , ascending=False)

,importance,feature
2,28.944791,title_ctr
5,14.718660,title_query_cosine_similarity
0,5.156430,title_sum
23,4.865906,title_nunique_prefix
1,4.820272,title_count
17,4.599051,q_t_common_words
3,3.399962,title_query_dot_similarity
39,2.893822,longistStr_len
24,2.782175,0_lsi
22,2.556916,q_t_wc_ratio_unique


In [10]:
test_pre = ctb_model.predict_proba(test_features)[:,1]

In [20]:
sub = pd.read_csv("/home/kesci/input/bytedance/test_final_part1.csv",names = ['query_id','query','query_title_id','title'])

In [21]:
sub.head()

,query_id,query,query_title_id,title
0,1,1427 28 7825 602 1427,1,4767 36 29168 13423 15 3854 9206 28 27 602 782...
1,1,1427 28 7825 602 1427,2,28 602 130 7825\t
2,1,1427 28 7825 602 1427,3,308 28 602 7825 3542 557 27 52 175\t
3,1,1427 28 7825 602 1427,4,945 821 28 1539 27 7825 18744 107 131 146 22326\t
4,1,1427 28 7825 602 1427,5,602 7825 28 27 11483 15 855 38 257\t


In [22]:
sub['prediction'] = test_pre

In [23]:
sub[['query_id','query_title_id','prediction']].to_csv("first_zzp/result/sherry.csv",index = 0,header = 0)

In [24]:
sub[['query_id','query_title_id','prediction']].head()

,query_id,query_title_id,prediction
0,1,1,0.080736
1,1,2,0.156125
2,1,3,0.114137
3,1,4,0.357560
4,1,5,0.074332


In [25]:
!wget -O kesci_submit https://www.heywhale.com/kesci_submit&&chmod +x kesci_submit
!https_proxy="http://klab-external-proxy" ./kesci_submit -file /home/kesci/work/first_zzp/result/sherry.csv -token b1aa3315162ef0ed

wget: /opt/conda/lib/libcrypto.so.1.0.0: no version information available (required by wget)
wget: /opt/conda/lib/libssl.so.1.0.0: no version information available (required by wget)
wget: /opt/conda/lib/libssl.so.1.0.0: no version information available (required by wget)
--2019-07-28 15:43:31--  https://www.heywhale.com/kesci_submit
Resolving www.heywhale.com (www.heywhale.com)... 106.15.25.147
Connecting to www.heywhale.com (www.heywhale.com)|106.15.25.147|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6528405 (6.2M) [application/octet-stream]
Saving to: ‘kesci_submit’

kesci_submit        100%[===================>]   6.23M  18.8MB/s    in 0.3s    

2019-07-28 15:43:31 (18.8 MB/s) - ‘kesci_submit’ saved [6528405/6528405]

Kesci Submit Tool 3.0

> 已验证Token
> 提交文件 /home/kesci/work/first_zzp/result/sherry.csv (575242.66 KiB)
> 文件已上传        
> 提交完成
